In [ ]:
!pip install accelerate -U
!pip install transformers[torch] -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset

# Set seed for reproducibility
torch.manual_seed(42)

# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels) if self.labels is not None else len(self.encodings['input_ids'])

# Load data
train_df = pd.read_csv('human_values_annotation_groundtruth.csv', on_bad_lines='skip')
test_df = pd.read_csv('Anthropic_rejected_jun_24.csv', on_bad_lines='skip')

# Check for missing values in labels
if train_df['Label'].isnull().any():
    raise ValueError("Missing values in label column. Handle missing values before proceeding.")

# Convert labels to numeric format using LabelEncoder
label_encoder = LabelEncoder()
train_df['Label'] = label_encoder.fit_transform(train_df['Label'])
num_labels = len(label_encoder.classes_)

# Tokenize data using RoBERTa tokenizer with explicit max_length
max_length = 128  # Set your desired max length
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Ensure 'Text' column and 'rejected' column are of type `str`
train_texts = train_df['Text'].astype(str).tolist()
test_texts = test_df['rejected'].astype(str).tolist()

# Tokenize and encode the training texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
train_labels = torch.tensor(train_df['Label'].tolist(), dtype=torch.long)

# Tokenize and encode the augmentation dataset texts
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length)

# Create custom datasets
train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings)

# Fine-tune model using RoBERTa with regularization, data augmentation, and class weights
def model_init():
    model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_labels)
    return model

# Define training arguments with hyperparameter tuning
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=8,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=1000,
    weight_decay=0.01,
    logging_dir='./logs',
    save_strategy="epoch",
    save_total_limit=1,
    logging_steps=500,
    push_to_hub=False,
    disable_tqdm=False,
    gradient_accumulation_steps=2,
    fp16=True,
)

# Define trainer
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
)

# Train the model
print("Starting model training...")
trainer.train()
print("Model training completed.")

# Get predictions for the augmentation dataset
print("Making predictions on the augmentation dataset...")
predictions = trainer.predict(test_dataset)
print("Predictions made.")

# Get predicted labels
predicted_labels = torch.argmax(torch.from_numpy(predictions.predictions), dim=1).tolist()

# Convert the predicted labels back to human values
predicted_human_values = label_encoder.inverse_transform(predicted_labels)

# Create a DataFrame with the predictions and texts
pred_df = pd.DataFrame({'Text': test_df['rejected'], 'Prediction': predicted_human_values})

# Save predictions to csv
pred_df.to_csv('rejected_human_values_roberta.csv', index=False)
print('Saved results to rejected_human_values_roberta.csv')


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting model training...


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-10-26f861f76ef9>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Step,Training Loss
500,0.858600


<ipython-input-10-26f861f76ef9>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
<ipython-input-10-26f861f76ef9>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
<ipython-input-10-26f861f76ef9>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
<ipython-input-10-26f861f76ef9>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.c

Model training completed.
Making predictions on the augmentation dataset...


Predictions made.
Saved results to augmentation2_results_roberta.csv


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset

# Set seed for reproducibility
torch.manual_seed(42)

# Custom dataset class
class CustomDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels) if self.labels is not None else len(self.encodings['input_ids'])

# Load data
train_df = pd.read_csv('human_values_annotation_groundtruth.csv', on_bad_lines='skip')
test_df = pd.read_csv('Anthropic_chosen_jun_24.csv', on_bad_lines='skip')

# Check for missing values in labels
if train_df['Label'].isnull().any():
    raise ValueError("Missing values in label column. Handle missing values before proceeding.")

# Convert labels to numeric format using LabelEncoder
label_encoder = LabelEncoder()
train_df['Label'] = label_encoder.fit_transform(train_df['Label'])
num_labels = len(label_encoder.classes_)

# Tokenize data using RoBERTa tokenizer with explicit max_length
max_length = 128  # Set your desired max length
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Ensure 'Text' column and 'rejected' column are of type `str`
train_texts = train_df['Text'].astype(str).tolist()
test_texts = test_df['chosen'].astype(str).tolist()

# Tokenize and encode the training texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
train_labels = torch.tensor(train_df['Label'].tolist(), dtype=torch.long)

# Tokenize and encode the augmentation dataset texts
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length)

# Create custom datasets
train_dataset = CustomDataset(train_encodings, train_labels)
test_dataset = CustomDataset(test_encodings)

# Fine-tune model using RoBERTa with regularization, data augmentation, and class weights
def model_init():
    model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=num_labels)
    return model

# Define training arguments with hyperparameter tuning
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=8,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=1000,
    weight_decay=0.01,
    logging_dir='./logs',
    save_strategy="epoch",
    save_total_limit=1,
    logging_steps=500,
    push_to_hub=False,
    disable_tqdm=False,
    gradient_accumulation_steps=2,
    fp16=True,
)

# Define trainer
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
)

# Train the model
print("Starting model training...")
trainer.train()
print("Model training completed.")

# Get predictions for the augmentation dataset
print("Making predictions on the augmentation dataset...")
predictions = trainer.predict(test_dataset)
print("Predictions made.")

# Get predicted labels
predicted_labels = torch.argmax(torch.from_numpy(predictions.predictions), dim=1).tolist()

# Convert the predicted labels back to human values
predicted_human_values = label_encoder.inverse_transform(predicted_labels)

# Create a DataFrame with the predictions and texts
pred_df = pd.DataFrame({'Text': test_df['chosen'], 'Prediction': predicted_human_values})

# Save predictions to csv
pred_df.to_csv('chosen_human_values_roberta.csv', index=False)
print('Saved results to chosen_human_values_roberta.csv')


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting model training...


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-11-0c8a1134b4d4>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])


Step,Training Loss
500,0.858600


<ipython-input-11-0c8a1134b4d4>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
<ipython-input-11-0c8a1134b4d4>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
<ipython-input-11-0c8a1134b4d4>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item['labels'] = torch.tensor(self.labels[idx])
<ipython-input-11-0c8a1134b4d4>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.c

Model training completed.
Making predictions on the augmentation dataset...


Predictions made.
Saved results to augmentation2_results_roberta.csv
